### Setup

Update the url and run the block to update the weather api spec

In [ ]:
# use a url parser to get hostname, path and query strings
url = ""

from urllib.parse import urlparse, parse_qs
import json

# Parse the URL
parsed_url = urlparse(url)
# Extract the hostname, path, and query string
hostname = parsed_url.hostname
path = parsed_url.path  
query_string = parsed_url.query
# Parse the query string into a dictionary
query_dict = parse_qs(query_string)
# Convert the dictionary to a JSON string
query_json = json.dumps(query_dict, indent=4)
# Print the results
print("Hostname:", hostname)
print("Path:", path)
print("Query String:", query_string)
print("Query Dictionary:", query_dict)

# read json file
import json
import os   

with open("conf/weather_openapi_template.json", "r") as f:
    txt = f.read()
    txt = txt.replace("{path}", path)
    txt = txt.replace("{hostname}", hostname)
    # Parse query_json back into a dictionary
    query_dict_parsed = json.loads(query_json)
    
    txt = txt.replace("{sig}", query_dict_parsed["sig"][0])
    txt = txt.replace("{sp}", query_dict_parsed["sp"][0])

    #update file
    with open("conf/weather_openapi.json", "w") as f:
        f.write(txt)






In [ ]:
# Import packages
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv

load_dotenv(override=True) # Load environment variables from .env file

# Use the connection string to connect to your Foundry project
project_connection_string = os.getenv("AIPROJECT_CONNECTION_STRING")
try:
    project = AIProjectClient.from_connection_string(
        conn_str=project_connection_string, credential=DefaultAzureCredential()
    )
except Exception as e:
    print(f"Failed to connect to project: {e}")
    raise



In [ ]:
import json
from azure.ai.projects.models import OpenApiAnonymousAuthDetails, OpenApiTool

with open("conf/weather_openapi.json", "r") as f:
    openapi_weather = json.loads(f.read())

# Create Auth object for the OpenApiTool (note that connection or managed identity auth setup requires additional setup in Azure)
auth = OpenApiAnonymousAuthDetails()

# Initialize agent OpenApi tool using the read in OpenAPI spec
openapi_tool = OpenApiTool(
    name="get_weather",
    spec=openapi_weather,
    description="Retrieve weather information for a location",
    auth=auth
)


### Create a simple agent

In [ ]:
## need to login using the command azd auth login
agent = project.agents.create_agent(
        model=os.getenv("CHAT_MODEL"),
        name="LogicAppAgent",
        instructions="You are helpful AI assistant.",
        tools=openapi_tool.definitions,
        tool_resources=openapi_tool.resources
    )
print(f"Created agent, agent ID: {agent.id}")

### Create a thread for conversation presistance

In [ ]:
# Create a thread
thread = project.agents.create_thread()
print(f"Created thread, thread ID: {thread.id}")

In [ ]:
def sendMessage(msg):
    # Get messages from the thread
    message = project.agents.create_message(
            thread_id=thread.id,
            role="user",
            content=msg
        )
    print(f"Created message, ID: {message.id}")

    run = project.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
            print(f"Run failed: {run.last_error}")
            raise Exception(f"Run failed: {run.last_error}")

    messages = project.agents.list_messages(thread_id=thread.id, order="asc", after=message.id)
    for msg in messages.data:
        role = msg.role
        content = msg.content[0].text.value
        print(f"{role.capitalize()}: {content}")


### Get the response

In [ ]:
sendMessage("What is the weather in Bombay?")